In [ ]:
# Creating one-hot encodings from the dataset labels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# dict contains file paths to tensorflow datasets created using preprocessing code
saved_data_paths = {
    "test4": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/wangtest",
    "wangtrain":"/content/drive/Shareddrives/DL_Final_Project/tf_datasets/wangtrain",

    "test1": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/test1",
    "test2": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/test2",
    "test3": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/test3",

    "train1a": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train1a",
    "train1b": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train1b",
    "train1c": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train1c",

    "train2a": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train2a",
    "train2b": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train2b",
    "train2c": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train2c",

    "train3a": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train3a",
    "train3b": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train3b",
    "train3c": "/content/drive/Shareddrives/DL_Final_Project/tf_datasets/train3c"
}

In [ ]:
# paths to save onehot datasets to:
onehot_datapaths2 = {
    "test1": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/test1",
    "test2": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/test2",
    "test3": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/test3",
    "test4": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/test4",

    "train1": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/train1",
    "train2": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/train2",
    "train3": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/train3",
    "train4": "/content/drive/Shareddrives/DL_Final_Project/tf_data_onehot2/train4"
}

In [ ]:
def load_dataset(dataset_name: str):
  path = saved_data_paths[dataset_name]
  dataset = tf.data.Dataset.load(path)
  print("Dataset", dataset_name, "loaded")
  return dataset

In [ ]:
num_classes_dict = {
    "train1": 1777,
    "train2": 1656,
    "train3": 1628,
    "train4": 1588
}

In [ ]:
def transform_labels(labels, num_classes):
    # Get unique labels and number of classes
    unique_labels = sorted(set(labels))

    # Map each unique label to an integer in the range [0, num_classes - 1]
    label_map = {str(label): index for index, label in enumerate(unique_labels)}

    print("Labels transformed")
    return label_map

In [ ]:
import tensorflow as tf
import os
import json
from tqdm import tqdm

def get_one_hot_train_dataset(dataset_name, dataset, json_file):
    label_one_hot_dict = {}
    labels = []
    for item in dataset:
        _, label = item
        labels.append(label.numpy())

    num_classes = num_classes_dict[dataset_name]
    transformed_label_dict = transform_labels(labels, num_classes)

    # Create a directory for saving batches if it doesn't exist
    directory = '/content/drive/Shareddrives/DL_Final_Project/batches_tf'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Iterate over the dataset and save each batch
    batch_size = 5000
    current_batch = 0
    num_batches = len(dataset) // batch_size
    remaining_samples = len(dataset) % batch_size
    with tqdm(total=num_batches, desc="Saving batches") as pbar:
        for batch_images, batch_labels in dataset.batch(batch_size):
            # Prepare the batch data
            batch_data = []
            for image, label in zip(batch_images, batch_labels):
                one_hot_encoding = tf.one_hot(
                    transformed_label_dict[str(tf.get_static_value(label))], num_classes)
                batch_data.append((image, one_hot_encoding))

            # Convert the batch data to TensorFlow Dataset
            batch_tf_dataset = tf.data.Dataset.from_generator(
                lambda: batch_data,
                output_signature=(
                    tf.TensorSpec(shape=batch_images.shape[1:], dtype=batch_images.dtype),
                    tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)
                ))

            # Save the batch as a TF dataset
            batch_file_path = os.path.join(directory, f"{dataset_name}_batch_{current_batch}.tfdataset")
            batch_tf_dataset.save(batch_file_path)
            pbar.update(1)
            current_batch += 1

    print("Batches saved")

    # # Save labels and their one-hot encodings to JSON
    # json_path = os.path.join(directory, json_file)
    # with open(json_path, 'w') as json_file:
    #     json.dump(label_one_hot_dict, json_file)
    # print("JSON written")

    return transformed_label_dict, num_classes


In [ ]:
def get_one_hot_test_dataset(dataset_name, transformed_label_dict, num_classes, dataset):
    new_dataset = []
    for image, label in dataset:
        one_hot_encoding = tf.one_hot(transformed_label_dict[str(tf.get_static_value(label))], num_classes)
        new_dataset.append((image, one_hot_encoding))

    # Convert the new dataset to TensorFlow Dataset
    tf_dataset = tf.data.Dataset.from_generator(lambda: new_dataset, output_signature=(
        tf.TensorSpec(shape=image.shape, dtype=image.dtype),
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)
    ))

    return tf_dataset

In [ ]:
concat_save_path = {
    "train1": "/content/drive/Shareddrives/DL_Final_Project/concat_datasets/train1",
    "train2": "/content/drive/Shareddrives/DL_Final_Project/concat_datasets/train2",
    "train3": "/content/drive/Shareddrives/DL_Final_Project/concat_datasets/train3",
    "train4": "/content/drive/Shareddrives/DL_Final_Project/concat_datasets/train4"
}

In [ ]:
# ======================= LOAD DATASETS =======================
'''
save training data
@train_name: name of training set
@test_name: name of test set
'''
def save_group(train_name, test_name):
  print("========= TRAIN SET ========")
  print(train_name)
  print("----------------------------")

  # ================== LOADING DATASETS ==================
  train_dataset = tf.data.Dataset.load(concat_save_path[train_name])
  print(train_name, "concat loaded")

  # ================== ONEHOT ENCODING ==================
  trans_dict, num_classes = get_one_hot_train_dataset(train_name, train_dataset, train_name+".json")
  print("one hot data loaded")
  print(train_name, "num classes:", num_classes)

  # ================== SAVING DATASET TOGETHER ==================
  print(train_name, "saved")

  # ================== DONE ==================
  print(train_name, "successfully processed!")
  # -----------------------------------------------------
  print("========= TEST SET =========")
  print(test_name)
  print("----------------------------")
  # ================== LOADING DATASET ==================
  test_dataset = load_dataset(test_name)

  # ================== ONEHOT ENCODING ==================
  onehot_testdataset = get_one_hot_test_dataset(test_name, trans_dict, num_classes, test_dataset)
  print(test_name, "one hot created")

  # ================== SAVING DATASET TOGETHER ==================
  onehot_testdataset.save(onehot_datapaths2[test_name])
  print(test_name, "saved")

  # ================== DONE ==================
  print(test_name, "successfully processed!")


In [ ]:
# save_group("train1", "test1")
# save_group("train2", "test2")
# save_group("train3", "test3")
save_group("train4", "test4")

========= TRAIN SET ========
train4
----------------------------
train4 concat loaded
Labels transformed


Saving batches: 13it [05:22, 24.77s/it]


Batches saved
one hot data loaded
train4 num classes: 1588
train4 saved
train4 successfully processed!
========= TEST SET =========
test4
----------------------------
Dataset test4 loaded
test4 one hot created
test4 saved
test4 successfully processed!
